In [1]:
import os
from dashscope import MultiModalConversation
from http import HTTPStatus
from PIL import Image
import dashscope
import random
import requests
import re
from tqdm import tqdm
import numpy as np
dashscope.api_key = 'sk-fc58a205ddfb4a718bf904619f3aba59'
# os.system('export DASHSCOPE_API_KEY=)
def extract_box(text, w, h):
    pattern = r'\((.*?)\)'
    matches = re.findall(pattern, text)
    box = []
    for match in matches:
        box += match.split(',')
    for i in range(len(box)):
        box[i] = eval(box[i])
    box[0] = int(box[0] / 1000 * w)
    box[1] = int(box[1] / 1000 * h)
    box[2] = int(box[2] / 1000 * w)
    box[3] = int(box[3] / 1000 * h)
    return box
def reasoning_grouding_by_qwen(file_path, text_prompt):
    """Sample of use local file.
       linux&mac file schema: file:///home/images/test.png
       windows file schema: file://D:/images/abc.png
    """

    messages = [{
        'role': 'system',
        'content': [{
            'text': '''
                    You are an AI assistant who is good at making accurate vision grounding based on questions asked
                    '''
        }]
    }, {
        'role':
        'user',
        'content': [
            {
                'image': f'file://{file_path}'
            },
            {
                'text': text_prompt
            },
        ]
    }]
    response = MultiModalConversation.call(model='qwen-vl-chat-v1', messages=messages)
    image = Image.open(file_path)
    
    # print(response.output.choices)
    # answer = response.output.choices[0].message.content[0]['box']
    answer = response.output.choices[0].message.content
    # result_image = response.output.choices[0].message.content[1]['result_image']
    # result_image = Image.open(requests.get(result_image, stream=True).raw)
    box = extract_box(answer, *(image.size))
    return answer, box

In [2]:
from segment_anything import sam_model_registry, SamPredictor

sam_checkpoint = "ckpts/sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

In [6]:
# file_path = 'data/nerf_llff_data/room/images_4/DJI_20200226_143850_006.JPG'
# file_path = 'data/nerf_llff_data/fern/images_4/IMG_4026.JPG'
# prompt = ''
# text_prompt = 
# answer, box = reasoning_grouding_by_qwen(file_path, text_prompt)

In [17]:
# scene = 'fern'
# prompt = 'the biggest tree'
# scene = 'flower'
# prompt = 'what looks the most colorful'
# scene = 'fortress'
# prompt = 'what seems to be made of metal'
scene = 'horns'
prompt = 'which is the biggest skeleton'
# prompt = 'what can be used to play videos'

In [18]:
images_root = f'data/nerf_llff_data/{scene}/images_4'
image_names = os.listdir(images_root)
image_names = [image_name for idx, image_name in enumerate(image_names) if idx % 8 == 0]
image_paths = [os.path.join(images_root, image_name) for image_name in image_names]

text_prompt = f'Please grounding <ref> {prompt} </ref>'
save_path = f'output/qwen_sam/{scene}/{prompt}'
os.makedirs(save_path, exist_ok=True)
# image_names = os.listdir(images_root)
# image_paths = [os.path.join(images_root, name) for name in image_names]
# images = [Image.open(path) for path in image_paths]


In [19]:
image_names

['IMG_1800.JPG',
 'IMG_1808.JPG',
 'IMG_1816.JPG',
 'IMG_1824.JPG',
 'IMG_1832.JPG',
 'IMG_1840.JPG']

In [20]:
for i in tqdm(range(len(image_paths))):
    image_name = image_names[i]
    image_path = image_paths[i]
    image = Image.open(image_path)
    answer, box = reasoning_grouding_by_qwen(image_path, text_prompt)
    box = np.array(box)
    # result_list, mask_result_list, mask_list, mask_rgb_list, output_str = lisa_pipeline(lisa_text_prompt, image=image)
    predictor.set_image(np.array(image))
    masks, _, _ = predictor.predict(
    point_coords=None,
    point_labels=None,
    box=box[None, :],
    multimask_output=False,
)
    save_name = image_name.split('.')[0] + '.png'
    Image.fromarray(masks[0]).save(os.path.join(save_path, save_name))
    

100%|██████████| 6/6 [00:14<00:00,  2.42s/it]
